In [ ]:
%cd /content/drive/MyDrive/Project B/DatasetCondensation-master
sys.path.append('/content/drive/MyDrive/Project B/DatasetCondensation-master')

In [ ]:
import sys
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import scipy.io as so

/content/drive/MyDrive/Project B/DatasetCondensation-master


# creating condenced dataset on MNIST and CIFAR10 

In [ ]:
!python Main_colab.py --save_path 'result_real_init_MNIST' --num_eval 100 --epoch_eval_train 300 --Iteration 1000 --ipc 10 --dataset MNIST --init real 

In [ ]:
!python Main_colab.py --save_path 'result_real_init_MNIST' --num_eval 100 --epoch_eval_train 300 --Iteration 1000 --ipc 10 --dataset MNIST --init noise 

In [ ]:
!python Main_colab.py --save_path 'result_real_init_CIFAR10' --num_eval 100 --epoch_eval_train 300 --Iteration 1000 --ipc 10 --dataset CIFAR10 --init real 

In [ ]:
!python Main_colab.py --save_path 'result_real_init_CIFAR10' --num_eval 100 --epoch_eval_train 300 --Iteration 1000 --ipc 10 --dataset CIFAR10 --init noise 

# Cross-architecture test

In [ ]:
!python Main_colab.py --save_path 'Result_Cross_architecuture' --num_eval 50 --epoch_eval_train 100 --Iteration 100 --eval_mode M

# Creating model on the original dataset

In [ ]:
import os
import time
import copy
import argparse
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug



In [ ]:
parser = argparse.ArgumentParser(description='Parameter Processing')
parser.add_argument('--dataset', type=str, default='CIFAR10', help='dataset')
parser.add_argument('--model', type=str, default='ConvNet', help='model')
parser.add_argument('--data_path', type=str, default='data', help='dataset path')
parser.add_argument('--save_path', type=str, default='result', help='path to save results')
##################################################
parser.add_argument("-f", "--file", required=False) # this particular code is essential for the parser to work in google colab
##################################################
args = parser.parse_args()
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/DIP_Assignment_v5/Project B/DatasetCondensation-master')
# from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
import scipy.io as so

In [ ]:
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)
test_loader = torch.utils.data.DataLoader(dst_test, batch_size=256, shuffle=False, num_workers=0)
train_loader = torch.utils.data.DataLoader(dst_train, batch_size=256, shuffle=False, num_workers=0)

In [ ]:
def get_accuracy(model, dataloader):
    """
    computes the accuracy of the model
    """

    model.eval()
    device = next(model.parameters()).device
    accuracy = 0.0
    total = 0.0
    correct = 0.0

    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            # TODO
            # Return the accuracy
            # images = torch.unsqueeze(images,1)
            total += images.shape[0]
            logits =model(images)
            correct += (torch.max(logits, 1)[1].view(labels.size()).data == labels).float().sum()
        accuracy = 100*(correct/total)
    return accuracy

In [ ]:
model = get_network(args.model, channel, num_classes, im_size).to(args.device) # get a random model
model.train()
net_parameters = list(model.parameters())
optimizer_net = torch.optim.SGD(model.parameters(), lr=args.lr_net)  # optimizer_img for synthetic data
device = next(model.parameters()).device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
acc_hist = {'train':[], 'test': []}
for epoch in range(200):

    ## training step
    for i, (images, labels) in enumerate(train_loader):

        images = images.to(device)
        labels = labels.to(device)
        # images = torch.squeeze(images,1)
        # TODO
        # Follow the step in the tutorial
        ## forward + backprop + loss
        logits = model(images)
        loss = criterion(logits, labels)
        optimizer_net.zero_grad()
        loss.backward()

        ## update model params
        optimizer_net.step()
        model.eval()
        acc_hist['train'].append(get_accuracy(model, train_loader))
        acc_hist['test'].append(get_accuracy(model, test_loader))

        print('Epoch: %d | Train Accuracy: %.2f | Test Accuracy: %.2f'\
              %(epoch, acc_hist['train'][-1], acc_hist['test'][-1]))